In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
import tools

In [3]:
class Predictor(object):
    def __init__(self):
        """
        Initialize the values that are necessary for making predictions:
            temp_calendar: annual trend
            slope and intercept: of three day relationship
        """
        temps, year, month, day = self.get_data()
        self.temp_calendar = self.build_temp_calendar(
            temps, year, month, day)
        deseasonalized_temps = np.zeros(temps.size)
        for i, temp in enumerate(temps):
            seasonal_temp  = self.find_seasonal_temp(
                year[i], month[i], day[i])
            deseasonalized_temps[i] = temp - seasonal_temp

        self.slope, self.intercept = self.get_three_day_coefficients(
            deseasonalized_temps)

    def get_data(self):
        """
        Parameters
        ----------
        none

        Return
        ------
        temps: numpy.ndarray of dtype: float
        day, month, year: list containg 'str' objects
        """
        dataset = pd.read_csv('dataset.csv')
        # Date spiltting into day, month, year
        day = []
        month = []
        year = []

        for i in dataset['Date']:
            splitted = i.split('/')
            day.append(int(splitted[0]))
            month.append(int(splitted[1]))
            year.append(int(splitted[2]))

        # pandas.core.series.Series into numpy.ndarray of dtype: float
        temps = np.array(dataset['Temperature F'])
        return (temps, year, month, day)

    def build_temp_calendar(self, temps, year, month, day):
        """
        Create an array of typical temperatures by day-of-year.
        Day 0 = Jan 1, etc.

        Parameters
        ----------
        temps: array of floats
        year, month, day: array of ints

        Returns
        -------
        median_temp_calendar: array of floats of length 366
        """
        day_of_year = np.zeros(temps.size)
        for i_row in range(temps.size):
            day_of_year[i_row] = tools.find_day_of_year(
                year[i_row], month[i_row], day[i_row])

        ## Create 10-day medians for each day of the year.
        median_temp_calendar = np.zeros(366)
        for i_day in range(0, 365):
            low_day = i_day - 5
            high_day = i_day + 4
            if low_day < 0:
                low_day += 365
            if high_day > 365:
                high_day += -365
            if low_day < high_day:
                i_window_days = np.where(
                    np.logical_and(day_of_year >= low_day,
                                   day_of_year <= high_day))
            else:
                i_window_days = np.where(
                    np.logical_or(day_of_year >= low_day,
                                  day_of_year <= high_day))

            ten_day_median = np.median(temps[i_window_days])
            median_temp_calendar[i_day] = ten_day_median

            if i_day == 364:
                median_temp_calendar[365] = ten_day_median

        return median_temp_calendar


    def get_three_day_coefficients(self, residuals):
        """
        Parameters
        ----------
        residuals: array of floats

        Returns
        -------
        slope, intercept: floats
            Coefficients of the line showing the relationship
            between deseasonalized temperatures and those
            three dyas into the future.
        """
        slope, intercept = np.polyfit(residuals[:-3], residuals[3:], 1)
        return (slope, intercept)


    def find_seasonal_temp(self, year, month, day):
        """
        For a given day, month, and year, find the seasonal 
        high temperature for Fort Lauderdale Beach.

        Parameters
        ----------
        year, month, day: int
            The date of interest

        Returns
        -------
        seasonal_temp: float
        """
        doy = tools.find_day_of_year(year, month, day)
        seasonal_temp = self.temp_calendar[doy]
        return seasonal_temp


    def predict_deseasonalized(self, three_day_temp):
        """
        Based on a deseasonalized temperature, predict what the
        deseasonalized temperature will be three days in the future.

        Parameters
        ----------
        three_day_temp: float
            The measured temperature three days before the day of interest.

        Results
        -------
        predicted_temp: float
        """
        predicted_temp = self.intercept + self.slope * three_day_temp
        return predicted_temp


    def deseasonalize(self, temp, doy):
        """
        Deseasonalize a temperature by subtracting out the annual trend.

        Parameters
        ----------
        temp: float
        doy: int

        Return
        ------
        deseasonalized_temp: float
        """
        deseasonalized_temp = temp - self.temp_calendar[doy]
        return deseasonalized_temp


    def reseasonalize(self, deseasonalized_temp, doy):
        """
        Reconstitute the deseasonalized temperature by adding back in the annual trend.

        Parameters
        ----------
        deseasonalized_temp: float
        doy: int

        Return
        ------
        reseasonalized_temp: float
        """
        reseasonalized_temp = deseasonalized_temp + self.temp_calendar[doy]
        return reseasonalized_temp


    def predict(self, year, month, day, past_temp):
        """
        Parameters
        ----------
        year, month, day: ints
        past_temp: float
            The temperature from 3 days before the date of interest.
        """
        # Make a prediction for a single day.
        doy = tools.find_day_of_year(year, month, day)
        doy_past = doy - 3
        if doy_past < 0:
            doy_past += 365

        deseasonalized_temp = self.deseasonalize(past_temp, doy_past)
        # Make predictions for three days into the future.
        deseasonalized_prediction = self.predict_deseasonalized(
            deseasonalized_temp)
        prediction = self.reseasonalize(deseasonalized_prediction, doy)

        return prediction

In [4]:
def test():
    """
    Run through the data history, calculating the prediction
    error for each day.
    """
    # Create and initialize a new Predictor
    predictor = Predictor()
    temps, year, month, day = predictor.get_data()
    deseasonalized_temps = np.zeros(temps.size)
    doy = np.zeros(temps.size, dtype=np.int)
    for i, temp in enumerate(temps):
        seasonal_temp  = predictor.find_seasonal_temp(
            year[i], month[i], day[i])
        deseasonalized_temps[i] = temp - seasonal_temp
        doy[i] = tools.find_day_of_year(year[i], month[i], day[i])
    # Make predictions for three days into the future.
    deseasonalized_predictions = np.zeros(temps.size)
    for i, temp in enumerate(deseasonalized_temps):
        deseasonalized_predictions[i] = predictor.predict_deseasonalized(temp)

    predictions = np.zeros(temps.size - 3)
    for i, temp in enumerate(deseasonalized_predictions[:-3]):
        predictions[i] = predictor.reseasonalize(temp, doy[i + 3])

    residuals = temps[3:] - predictions
    print('MEA:', np.mean(np.abs(residuals)))
    
    actuals = temps[3:]
    # Compare predictions three days away vs. actuals for above/below 85.
    sensitivity = []
    targets = np.arange(84, 90)
    for target in targets:
        i_warm = np.where(actuals > 85)[0]
        i_warm_predictions = np.where(predictions > target)[0]
        n_true_positives = np.intersect1d(i_warm, i_warm_predictions).size
        n_false_negatives = np.setdiff1d(i_warm, i_warm_predictions).size
        n_false_positives = np.setdiff1d(i_warm_predictions, i_warm).size
        n_true_negatives = (actuals.size - n_true_positives -
                            n_false_positives - n_false_negatives)

        sensitivity.append(n_true_positives / (n_true_positives + n_false_positives))

In [5]:
import datetime

def fromToday_upto10day(test_temp, date="from today"):
    """
    This function predict temperatures upto 10 days
    from current date or from prescribed date.
    Parameters
    -----------
    test_temp: take current day temperature from user
    date: None(By default)
          otherwise; date is given in that format
          (i.e; '19-9-2018')
    
    Return 
    ------
    celcius: numpy array of floats that contain predicted
    temperatures in Celcius for 10 days
    """
    if date == "from today":
        current_date = datetime.date.today()    # 2019-12-20
    else:
        current_date = datetime.datetime.strptime(date, '%d-%m-%Y').date()  # <class 'datetime.date'>
 
    celcius = []
    temps = list()
    temps.insert(0, test_temp)
    print("Date       :  °F   : °C")
    print("-------------------------")
    for i in range(1, 11):    # foor 10 days
        future_date = current_date + datetime.timedelta(days=i) # current day + next day
        future = str(future_date).split("-")     # ['2019', '12', '20']
        prediction = predictor.predict(int(future[0]), # test_year = 2019
                                       int(future[1]), # test_month = 12
                                       int(future[2]), # test_day = 20
                                       temps[-1])  # taken last (or predicted) temerature
        temps.append(prediction)
        print(f"{future_date} : {str(prediction)[:4]}  : {str(tools.toCelcius(prediction))[:4]}")
        celcius.append(tools.toCelcius(prediction))
    return np.array(celcius)

In [6]:
if __name__ == '__main__':
    predictor = Predictor()
    test() 
    fromToday_upto10day(80, '1-1-2020') # return temps from given day upto 10 days later
    fromToday_upto10day(80)  # return temps from date upto 10 days

MEA: 2.090754003129972
Date       :  °F   : °C
-------------------------
2020-01-02 : 69.2  : 20.6
2020-01-03 : 64.5  : 18.1
2020-01-04 : 62.6  : 17.0
2020-01-05 : 62.2  : 16.7
2020-01-06 : 62.2  : 16.8
2020-01-07 : 62.2  : 16.8
2020-01-08 : 62.3  : 16.8
2020-01-09 : 62.4  : 16.9
2020-01-10 : 62.6  : 17.0
2020-01-11 : 62.6  : 17.0
Date       :  °F   : °C
-------------------------
2019-12-21 : 70.0  : 21.1
2019-12-22 : 66.0  : 18.9
2019-12-23 : 64.3  : 17.9
2019-12-24 : 63.6  : 17.6
2019-12-25 : 63.4  : 17.4
2019-12-26 : 63.4  : 17.4
2019-12-27 : 63.4  : 17.4
2019-12-28 : 63.4  : 17.4
2019-12-29 : 63.4  : 17.4
2019-12-30 : 63.3  : 17.4
